# Multiprocessing: Využití více jader CPU

Pokud vaše aplikace provádí těžké výpočty (zpracování obrazu, šifrování, data science), modul `threading` vám nepomůže. Musíte použít `multiprocessing` nebo `concurrent.futures.ProcessPoolExecutor`.

**Hlavní rozdíl:** Procesy nesdílejí paměť. Globální proměnná změněná v jednom procesu se v druhém nezmění!

## 1. ProcessPoolExecutor (Doporučený způsob)
Stejně jako u vláken máme "Pool", který se stará o vytváření procesů.

In [ ]:
import concurrent.futures
import time
import os

def vypocet_druhe_mocniny(x):
    # Simulace práce
    time.sleep(0.5)
    # Vypíšeme ID procesu (PID), abychom viděli, že to dělají různí dělníci
    pid = os.getpid()
    return f"Proces {pid} spočítal: {x}^2 = {x*x}"

def spust_pool():
    cisla = [1, 2, 3, 4, 5, 6, 7, 8]
    
    print(f"Hlavní proces má PID: {os.getpid()}")
    print("Startuji ProcessPoolExecutor...")
    
    # max_workers=None automaticky použije počet jader vašeho CPU
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # map rozdělí práci mezi procesy
        vysledky = executor.map(vypocet_druhe_mocniny, cisla)

        for v in vysledky:
            print(v)

# V Jupyter Notebooku/interaktivním režimu to funguje rovnou,
# ale ve skriptu je nutné: if __name__ == "__main__":
if __name__ == "__main__":
    spust_pool()

## 2. Sdílení dat (Problém)
Podívejme se, co se stane, když se pokusíme použít globální proměnnou.

In [ ]:
import multiprocessing

seznam = []

def pridej_do_seznamu():
    global seznam
    seznam.append(1)
    print(f"Uvnitř procesu: {seznam}")

if __name__ == "__main__":
    p = multiprocessing.Process(target=pridej_do_seznamu)
    p.start()
    p.join()
    
    print(f"V hlavním programu: {seznam}")
    print("Seznam je prázdný! Proces dostal jen kopii dat.")

Uvnitř procesu: [1]
V hlavním programu: []
Vidíte? Seznam je prázdný! Proces dostal jen kopii dat.


## 3. Jak sdílet data?
Musíme použít speciální objekty z `multiprocessing`.
* `Value`: Pro jedno číslo/hodnotu.
* `Array`: Pro pole čísel.
* `Manager`: Pro složitější struktury (list, dict), ale je pomalejší.

In [ ]:
def pridej_sdilene(sdileny_seznam):
    sdileny_seznam.append(1)
    print(f"Proces přidal prvek. Aktuálně: {sdileny_seznam}")

if __name__ == "__main__":
    # Manager se stará o synchronizaci dat
    with multiprocessing.Manager() as manager:
        sdileny_list = manager.list() # Speciální sdílený list
        
        p = multiprocessing.Process(target=pridej_sdilene, args=(sdileny_list,))
        p.start()
        p.join()
        
        print(f"V hlavním programu: {sdileny_list}")